## Installation de spark-cluster & findspark, pyspark 

In [6]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark 
findspark.init()
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("actions").setMaster("local[4]")
sc = SparkContext(conf = conf)

In [27]:
#Charger le dataset HousePrice.csv
#nettoyer le dataset en supprimant Heander

lines = sc.textFile("HousePrice.csv")
cleanedLines = lines.filter(lambda line: "date" not in line)

In [28]:
# récupérer le price(position 1) et la city (position 15)

City_Price_Rdd=cleanedLines.map(lambda line: (line.split(",")[15], float(line.split(",")[1])))

City_HouseNB_Rdd=cleanedLines.map(lambda line: (line.split(",")[15], 1))


In [30]:
# calculer le prix moyen pour chaque city
# utilisation les trabsformation union avec reduceByKey

City_Price_Total_Rdd=City_Price_Rdd.reduceByKey(lambda x,y: x+y)
City_HouseNB_Total_Rdd=City_HouseNB_Rdd.reduceByKey(lambda x,y: x+y)

AVG_By_City=City_Price_Total_Rdd.union(City_HouseNB_Total_Rdd).reduceByKey(lambda x,y: x/y)

AVG_By_City.collect()

[('Seattle', 579837.4672823235),
 ('Redmond', 667649.5347812298),
 ('Maple Valley', 336474.9118055521),
 ('North Bend', 399565.70666666003),
 ('Sammamish', 686917.5835323657),
 ('Auburn', 299340.44276556815),
 ('Bothell', 481441.8939393939),
 ('Federal Way', 289887.70398969593),
 ('Kirkland', 651583.5919080963),
 ('Woodinville', 609565.0217391305),
 ('Normandy Park', 506793.05555555556),
 ('Renton', 377040.96632902045),
 ('Duvall', 403994.12698411907),
 ('Kenmore', 447494.0404040303),
 ('Mercer Island', 1123817.6201550348),
 ('Black Diamond', 339605.55555555556),
 ('Ravensdale', 514071.4285714286),
 ('Algona', 207288.0),
 ('Tukwila', 308290.0689655172),
 ('Vashon', 472556.8965517241),
 ('Yarrow Point', 1194837.5),
 ('SeaTac', 245290.62068965516),
 ('Medina', 2046559.0909090908),
 ('Enumclaw', 307614.58333332144),
 ('Pacific', 225233.33333333334),
 ('Preston', 562450.0),
 ('Shoreline', 420392.3640469756),
 ('Kent', 439492.44464821625),
 ('Bellevue', 847180.6629953358),
 ('Lake Forest Pa

In [34]:
# Pour charque ville la surface moyenne (sqft_living) des maisons [4]
# utiliser la transformation join()

City_sqft_Rdd=cleanedLines.map(lambda line: (line.split(",")[15], float(line.split(",")[4])))
City_sqft_Total_Rdd=City_sqft_Rdd.reduceByKey(lambda x,y: x+y)

joinResult2=City_sqft_Total_Rdd.join(City_HouseNB_Total_Rdd)

def f(x): 
   sumBed=x[0]
   nb=x[1]
   return sumBed/nb
   
AVG_City_bed=joinResult2.mapValues(lambda x:f(x))
AVG_City_bed.collect()



[('Seattle', 1828.6236490781946),
 ('Redmond', 2491.7617021276596),
 ('Maple Valley', 2086.0416666666665),
 ('North Bend', 1995.4),
 ('Sammamish', 2830.12),
 ('Auburn', 2019.5795454545455),
 ('Bothell', 2319.3939393939395),
 ('Federal Way', 2054.114864864865),
 ('Kirkland', 2259.48128342246),
 ('Woodinville', 2663.008695652174),
 ('Normandy Park', 2093.277777777778),
 ('Renton', 2114.7610921501705),
 ('Duvall', 2161.5476190476193),
 ('Kenmore', 2110.530303030303),
 ('Mercer Island', 3275.313953488372),
 ('Black Diamond', 1863.3333333333333),
 ('Ravensdale', 2612.8571428571427),
 ('Algona', 1608.6),
 ('Tukwila', 1706.2068965517242),
 ('Vashon', 1889.6896551724137),
 ('Yarrow Point', 2472.5),
 ('SeaTac', 1678.5172413793102),
 ('Medina', 4103.636363636364),
 ('Enumclaw', 1922.5),
 ('Pacific', 1520.8333333333333),
 ('Preston', 2280.0),
 ('Shoreline', 1774.8373983739837),
 ('Kent', 1981.2702702702702),
 ('Bellevue', 2694.527972027972),
 ('Lake Forest Park', 2283.0555555555557),
 ('Des Moine